# ※ 참고 사이트- LSTM
https://seoyunion.tistory.com/18
https://dschloe.github.io/python/python_edu/07_deeplearning/deep_learning_lstm/
https://github.com/seoyunion/water-level-prediction/blob/main/Dam-water-level-prediction.ipynb

# 0.Setting

In [1]:
from glob import glob

import numpy as np
import pandas as pd

import matplotlib.pyplot as plt
import seaborn as sns

# loop측정
from tqdm import tqdm

import os
import warnings        
warnings.filterwarnings("ignore") 

%matplotlib inline

# 관련 라이브러리 임포트 
import matplotlib.font_manager as fm

#  한글글꼴로 변경
import matplotlib
# matplotlib.rcParams["font.family"]="Malgun Gothic"

# 그래프에서 마이너스 폰트 깨지는 문제에 대한 대처
matplotlib.rcParams['axes.unicode_minus'] = False

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import RobustScaler
from sklearn.preprocessing import MinMaxScaler

from keras.models import Sequential
from keras.layers import Dropout,Dense, SimpleRNN
from keras.callbacks import EarlyStopping, ModelCheckpoint
from keras.layers import LSTM

# 1. Data Preprocessing

## 1-1) Water Data

In [3]:
w_list = sorted(glob("./competition_data/water_data/*.csv"))
w_list

[]

In [7]:
water_df=pd.DataFrame()

for i in w_list :
    
    w_df = pd.read_csv(i)
    w_df = w_df.replace(" ", np.nan)
    w_df = w_df.interpolate(method = 'values')
    w_df = w_df.fillna(0)
    water_df= pd.concat([water_df, w_df])

water_df.head()

,ymdhm,swl,inf,sfw,ecpc,tototf,tide_level,wl_1018662,fw_1018662,wl_1018680,fw_1018680,wl_1018683,fw_1018683,wl_1019630,fw_1019630
0,2012-05-01 00:00,24.800,555.0,219.07,24.93,555.0,445.0,310.7,469.05,300.2,0.0,290.0,729.80,275.3,540.18
1,2012-05-01 00:10,24.794,464.6,218.86,25.15,562.9,449.0,314.7,498.00,300.2,0.0,290.0,731.48,275.3,540.18
2,2012-05-01 00:20,24.789,478.1,218.69,25.31,576.4,451.0,313.7,490.68,301.2,0.0,290.0,726.42,275.3,540.18
3,2012-05-01 00:30,24.789,464.8,218.69,25.31,563.1,452.0,311.7,476.21,301.2,0.0,290.0,726.42,276.3,552.17
4,2012-05-01 00:40,24.789,478.1,218.69,25.31,576.4,450.0,311.7,476.21,301.2,0.0,291.0,707.17,277.3,564.29


In [11]:
# df.iloc[:,0] == df['ymdhm']
def object_to_datetime(df,format):
    df.iloc[:,0] = pd.to_datetime(df.iloc[:,0], format=format)
    return df

In [12]:
water_df = object_to_datetime(df=water_df,format='%Y-%m-%d %H:%M')
water_df.set_index("ymdhm", inplace=True)
water_df.head()

,swl,inf,sfw,ecpc,tototf,tide_level,wl_1018662,fw_1018662,wl_1018680,fw_1018680,wl_1018683,fw_1018683,wl_1019630,fw_1019630
ymdhm,,,,,,,,,,,,,,
2012-05-01 00:00:00,24.800,555.0,219.07,24.93,555.0,445.0,310.7,469.05,300.2,0.0,290.0,729.80,275.3,540.18
2012-05-01 00:10:00,24.794,464.6,218.86,25.15,562.9,449.0,314.7,498.00,300.2,0.0,290.0,731.48,275.3,540.18
2012-05-01 00:20:00,24.789,478.1,218.69,25.31,576.4,451.0,313.7,490.68,301.2,0.0,290.0,726.42,275.3,540.18
2012-05-01 00:30:00,24.789,464.8,218.69,25.31,563.1,452.0,311.7,476.21,301.2,0.0,290.0,726.42,276.3,552.17
2012-05-01 00:40:00,24.789,478.1,218.69,25.31,576.4,450.0,311.7,476.21,301.2,0.0,291.0,707.17,277.3,564.29


## 1-2) RainFall Data

In [5]:
rf_list = sorted(glob("competition_data/rf_data/*.csv"))
rf_list

['competition_data/rf_data\\rf_2012.csv',
 'competition_data/rf_data\\rf_2013.csv',
 'competition_data/rf_data\\rf_2014.csv',
 'competition_data/rf_data\\rf_2015.csv',
 'competition_data/rf_data\\rf_2016.csv',
 'competition_data/rf_data\\rf_2017.csv',
 'competition_data/rf_data\\rf_2018.csv',
 'competition_data/rf_data\\rf_2019.csv',
 'competition_data/rf_data\\rf_2020.csv',
 'competition_data/rf_data\\rf_2021.csv',
 'competition_data/rf_data\\rf_2022.csv']

In [8]:
rainfall_df=pd.DataFrame()

for i in rf_list :
    
    w_df = pd.read_csv(i)
    w_df = w_df.replace(" ", np.nan)
    w_df = w_df.interpolate(method = 'values')
    w_df = w_df.fillna(0)
    rainfall_df= pd.concat([rainfall_df, w_df])

rainfall_df.head()

,ymdhm,rf_10184100,rf_10184110,rf_10184140
0,2012-05-01 00:00,0.0,0.0,0.0
1,2012-05-01 00:10,0.0,0.0,0.0
2,2012-05-01 00:20,0.0,0.0,0.0
3,2012-05-01 00:30,0.0,0.0,0.0
4,2012-05-01 00:40,0.0,0.0,0.0


In [13]:
rainfall_df = object_to_datetime(df=rainfall_df,format='%Y-%m-%d %H:%M')
rainfall_df.set_index("ymdhm", inplace=True)
rainfall_df.head()

,rf_10184100,rf_10184110,rf_10184140
ymdhm,,,
2012-05-01 00:00:00,0.0,0.0,0.0
2012-05-01 00:10:00,0.0,0.0,0.0
2012-05-01 00:20:00,0.0,0.0,0.0
2012-05-01 00:30:00,0.0,0.0,0.0
2012-05-01 00:40:00,0.0,0.0,0.0


## 1-3) Sample Submission Data

In [10]:
sample=pd.read_csv("competition_data/sample_submission.csv")
sample.head()

,ymdhm,wl_1018662,wl_1018680,wl_1018683,wl_1019630
0,2022-06-01 00:00,0,0,0,0
1,2022-06-01 00:10,0,0,0,0
2,2022-06-01 00:20,0,0,0,0
3,2022-06-01 00:30,0,0,0,0
4,2022-06-01 00:40,0,0,0,0


In [14]:
sample = object_to_datetime(df=sample,format='%Y-%m-%d %H:%M')
sample.set_index("ymdhm", inplace=True)
sample.head()

,wl_1018662,wl_1018680,wl_1018683,wl_1019630
ymdhm,,,,
2022-06-01 00:00:00,0,0,0,0
2022-06-01 00:10:00,0,0,0,0
2022-06-01 00:20:00,0,0,0,0
2022-06-01 00:30:00,0,0,0,0
2022-06-01 00:40:00,0,0,0,0


## 1-4) 데이터 시간대 확인

In [20]:
water_df.name = "water_data"
rainfall_df.name = "rain_data"
sample.name = "submission"

In [21]:
# 데이터 시간대 확인하기
def check_datetime(df):
    print(df.name)
    print(df.select_dtypes('datetime64[ns]').head(1).index[0])
    print(df.select_dtypes('datetime64[ns]').tail(1).index[0])
    print("-"*30)
    return None


check_datetime(water_df)
check_datetime(rainfall_df)
check_datetime(sample)

water_data
2012-05-01 00:00:00
2022-07-18 23:50:00
------------------------------
rain_data
2012-05-01 00:00:00
2022-07-18 23:50:00
------------------------------
submission
2022-06-01 00:00:00
2022-07-18 23:50:00
------------------------------


In [19]:
water_cp =water_df.copy()
rain_cp =rainfall_df.copy()

## 1-5) data target 분리

In [23]:
## df_water 에서 '해당하는 컬럼'의 값만 추출
## 'sample.columns' == ['wl_1018662', 'wl_1018680', 'wl_1018683', 'wl_1019630']
## ['청담대교 수위', '잠수교 수위', '한강대교 수위', '행주대교 수위']
target = water_cp.loc[:,sample.columns]

## df_water 에서 '해당하는 컬럼'을 제외하고 값 추출
## df_rf와 행 방향(가로 방향, axis=1) 으로 concat 
data = pd.concat((rain_cp, water_cp.drop(sample.columns,axis=1)),axis=1)

## 1-6) data와 target 하나 밀어주기 (과거데이터를 사용해야 함으로)

In [25]:
_target = target.reset_index(drop=True) ## ['청담대교 수위', '잠수교 수위', '한강대교 수위', '행주대교 수위']
_data = data.reset_index(drop=True)

## index가 1부터 시작하도록
_data.index += 1  

tot=pd.concat((_data,_target),axis=1)
tot=tot.sort_index()

## tot의 첫 번째 행(0번째 인덱스)이, 위의 [4개 컬럼]을 제외하고는 모두 NaN 값.
## 행은 1번째 인덱스부터 끝까지 : 열은 -1번째 인덱스까지(열은 모두) 데이터 추출
tot=tot.iloc[1:-1] 

## sample.columns == ['wl_1018662', 'wl_1018680', 'wl_1018683', 'wl_1019630']
## ['청담대교 수위', '잠수교 수위', '한강대교 수위', '행주대교 수위']
target = tot.loc[:,sample.columns]
data = tot.drop(sample.columns,axis=1)

In [26]:
target.head()

,wl_1018662,wl_1018680,wl_1018683,wl_1019630
1,314.7,300.2,290.0,275.3
2,313.7,301.2,290.0,275.3
3,311.7,301.2,290.0,276.3
4,311.7,301.2,291.0,277.3
5,311.7,301.2,291.0,277.3


In [27]:
data.head()

,rf_10184100,rf_10184110,rf_10184140,swl,inf,sfw,ecpc,tototf,tide_level,fw_1018662,fw_1018680,fw_1018683,fw_1019630
1,0.0,0.0,0.0,24.800,555.0,219.07,24.93,555.0,445.0,469.05,0.0,729.80,540.18
2,0.0,0.0,0.0,24.794,464.6,218.86,25.15,562.9,449.0,498.00,0.0,731.48,540.18
3,0.0,0.0,0.0,24.789,478.1,218.69,25.31,576.4,451.0,490.68,0.0,726.42,540.18
4,0.0,0.0,0.0,24.789,464.8,218.69,25.31,563.1,452.0,476.21,0.0,726.42,552.17
5,0.0,0.0,0.0,24.789,478.1,218.69,25.31,576.4,450.0,476.21,0.0,707.17,564.29


### - len(submission) == 6912 (2022-06-01 ~ 2022-07-18까지의 데이터)

In [29]:
## 최근 6912행(밑에서부터 위로 6912개의 행)은 제외하고, 데이터 추출
train_target=target.iloc[:-len(sample),:]

## 최근 6912행만 추출
test_target=target.iloc[-len(sample):,:]

train_data=data.iloc[:-len(sample),:]
test_data=data.iloc[-len(sample):,:]

In [35]:
# train_target.isnull().sum()
# test_target.isnull().sum()
# train_data.isnull().sum()
# test_data.isnull().sum()

In [36]:
print('--data--')
print(train_data.shape)
print(test_data.shape)
print('--target--')
print(train_target.shape)
print(test_target.shape)

--data--
(269423, 13)
(6912, 13)
--target--
(269423, 4)
(6912, 4)


# 2. Modeling
- 시계열 예측 지표 (MSE, MAE, RMSE, MAPE)
> - https://biology-statistics-programming.tistory.com/59
> - https://mizykk.tistory.com/102 \
> ① MSE(Mean Squared Error): SSE를 자유도로 나누어준 값이다. 자유도를 나누어 줌으로써 MSE가 크다의 의미는 단순히 데이터가 많아서 크다는 것이 아니라(SSE는 데이터 수가 많아지면 커진다.) 실제 차이가 커서 크다고 판단할 수 있게 되는 것이다.\
> ② MAE(Mean Absolute Error): 실제 관측값과 회귀식으로 예측한 값 차이를 절대값의 합을 해준 것이다. MSE가 R2 Norm을 사용했더라면 MAE는 R1 Norm을 사용한 것이라고 할 수 있다.\
> ③ RMSE(Root Mean Squared error): MSE에 루트를 씌워준 값이다.\
> ④ MAPE(Mean Absolute Percentage Error): MAE에서 실제 관측값과 회귀식으로 예측한 값 차이를 회귀식으로 예측한 값으로 나눈 절대값의 합이다. MAE는 outlier에 취약할 수 있는데(한 없이 커질 수 있다는 의미이다.) MAPE는 최대 크기가 제한되므로 이를 보완한 개념이라고 볼 수 있다.

In [40]:
tmp=pd.DataFrame()
for i in range(test_target.shape[1]): ## test_target.shape[1] == 4
    #train/test size 설정
    train_size = int(len(train_data)*0.6)
    validation_size = int(len(train_data)*0.3)+train_size

    #train/test 학습 및 라벨 설정
    #종가를 예측하기 위해 종가를 label로 설정
    train_x = np.array(train_data[:train_size])
    train_y = np.array(train_target.iloc[:train_size,i])

    validation_x =np.array(train_data[train_size:validation_size])
    validation_y = np.array(train_target.iloc[train_size:validation_size, i])

    test_x = np.array(test_data)
    test_y = np.array(test_target.iloc[:,i])
    
    learning_rate = 0.01
    training_cnt = 20
    batch_size = 200
    input_size = train_x.shape[1]

    time_step = 1

    # reshape into (size(개수), time step, 입력 feature)
    train_x = train_x.reshape(train_x.shape[0],1,input_size)
    validation_x = validation_x.reshape(validation_x.shape[0], 1, input_size)
    test_x = test_x.reshape(test_x.shape[0], 1, input_size)
    

        # 모델 구조
    model = Sequential()
    model.add(LSTM(512,input_shape=(1,input_size))) # 512는 다른 숫자로도 가능
    model.add(Dropout(0.2)) 
    model.add(Dense(1,activation='relu')) #output(target)은 '종가'이기 때문에 1요소 = Dense의 output레이어는 1로 설정

    #오차 및 최적화기 설정
    model.compile(loss='mse',optimizer='rmsprop',metrics=['mae','mape'])
    model.summary()


    #학습
    history = model.fit(train_x,train_y,epochs=training_cnt, batch_size=batch_size, verbose=1)
    val_mse, val_mae, val_mape = model.evaluate(test_x, test_y, verbose=0)
    
    pred = model.predict(test_x)
    column_name='col_'+str(i)
    tmp[column_name]=pd.DataFrame(pred)
    model_name='lstm_'+str(i)
    model.save(model_name+'.h5')

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 lstm_1 (LSTM)               (None, 512)               1077248   
                                                                 
 dropout (Dropout)           (None, 512)               0         
                                                                 
 dense (Dense)               (None, 1)                 513       
                                                                 
Total params: 1,077,761
Trainable params: 1,077,761
Non-trainable params: 0
_________________________________________________________________
Epoch 1/20
809/809 [==============================] - 12s 13ms/step - loss: 43386.9492 - mae: 182.3773 - mape: 54.2331
Epoch 2/20
809/809 [==============================] - 11s 13ms/step - loss: 3470.4036 - mae: 31.4054 - mape: 8.4235
Epoch 3/20
809/809 [==============================] - 11s 13ms/step - los

809/809 [==============================] - 11s 14ms/step - loss: 268.9614 - mae: 10.8101 - mape: 3.4072
Epoch 10/20
809/809 [==============================] - 11s 14ms/step - loss: 264.3896 - mae: 10.7806 - mape: 3.3995
Epoch 11/20
809/809 [==============================] - 11s 13ms/step - loss: 264.2100 - mae: 10.7678 - mape: 3.3974
Epoch 12/20
809/809 [==============================] - 11s 13ms/step - loss: 254.0376 - mae: 10.7098 - mape: 3.3834
Epoch 13/20
809/809 [==============================] - 11s 14ms/step - loss: 255.0901 - mae: 10.6989 - mape: 3.3782
Epoch 14/20
809/809 [==============================] - 12s 14ms/step - loss: 258.0569 - mae: 10.7414 - mape: 3.3906
Epoch 15/20
809/809 [==============================] - 11s 14ms/step - loss: 250.2503 - mae: 10.6159 - mape: 3.3521
Epoch 16/20
809/809 [==============================] - 11s 14ms/step - loss: 246.3140 - mae: 10.5657 - mape: 3.3397
Epoch 17/20
809/809 [==============================] - 11s 14ms/step - loss: 249.276

# 3. 제출 파일 만들기

In [42]:
tmp

,col_0,col_1,col_2,col_3
0,277.240540,265.099976,265.677307,269.314667
1,277.375549,268.330139,262.724731,265.608337
2,278.940247,270.067657,263.535767,263.795288
3,286.649963,275.431274,270.608307,264.493378
4,284.912842,271.453796,268.489105,260.100739
...,...,...,...,...
6907,289.508759,288.141602,287.633240,310.324432
6908,291.902954,287.194946,298.192322,306.162292
6909,291.363556,291.917603,307.155884,310.399963
6910,304.995300,298.315979,304.553040,311.414459


In [47]:
# tmp["col_0"]

In [48]:
sample_submission = pd.read_csv("competition_data/sample_submission.csv")

sample_submission["wl_1018662"] = tmp["col_0"]
sample_submission["wl_1018680"] = tmp["col_1"]
sample_submission["wl_1018683"] = tmp["col_2"]
sample_submission["wl_1019630"] = tmp["col_3"]

In [49]:
sample_submission.to_csv("baseline.csv", index = False)

# 4. 제출하기
- https://dacon.io/competitions/official/235949/mysubmission
> 본 대회는 점수가 **작을수록** 리더보드에서 높은 순위를 기록합니다.

<img src="img/220822_sub.png" width="600" height="800">